In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from scipy import stats
from copulae import StudentCopula, GaussianCopula

#DATA ACQUISITION
tickers = ['AAPL', 'MSFT', 'GOOGL']
print(f"Downloading data for: {tickers}...")
data = yf.download(tickers, start="2019-01-01", end="2024-01-01")

# Handle MultiIndex and extract Adjusted Close
if 'Adj Close' in data.columns:
    adj_close = data['Adj Close']
else:
    adj_close = data['Close']

# Calculate log returns
returns = np.log(adj_close / adj_close.shift(1)).dropna()

#VOLATILITY FILTERING (GARCH)
print("Fitting GARCH models to filter volatility clusters...")
std_residuals_dict = {}

for ticker in tickers:
    # Scale by 100 for numerical stability in the optimizer
    model = arch_model(returns[ticker] * 100, vol='Garch', p=1, q=1, dist='Normal')
    res = model.fit(disp='off')
    # Standardized Residuals = (Actual Returns / Estimated Volatility)
    std_residuals_dict[ticker] = res.resid / res.conditional_volatility

cleaned_data = pd.DataFrame(std_residuals_dict)

#MARGINAL DISTRIBUTIONS (FAT TAILS)
print("Modeling fat tails and transforming to Uniform [0,1]...")
dist_params = {}
uniform_data = pd.DataFrame()

for ticker in tickers:
    # Fit Student-t distribution to the GARCH residuals
    # Returns: (df, loc, scale)
    params = stats.t.fit(cleaned_data[ticker])
    dist_params[ticker] = params
    
    # Probability Integral Transform (PIT)
    # This maps the residuals into the [0, 1] range needed for Copulas
    uniform_data[ticker] = stats.t.cdf(cleaned_data[ticker], *params)

# STEP 4: COPULA CALIBRATION (CRASH CORRELATION)
print("Calibrating StudentCopula for tail dependence...")
# Initialize the copula. dim=3 for our 3 stocks.
cop = StudentCopula(dim=len(tickers))

# Fit the copula to your uniform data
# This captures the hidden correlation between assets during market crashes
cop.fit(uniform_data)


#MONTE CARLO SIMULATION
num_sim = 100000
print(f"Generating {num_sim} correlated scenarios...")

#Add .values to ensure we have a NumPy array for slicing
sim_uniform = cop.random(num_sim).values 

sim_residuals = pd.DataFrame()
for i, ticker in enumerate(tickers):
    df, loc, scale = dist_params[ticker]
    # Now sim_uniform[:, i] will work correctly
    sim_residuals[ticker] = stats.t.ppf(sim_uniform[:, i], df, loc, scale)

print("Scenarios generated. Proceeding to final price calculation...")

# Project future prices for a 1-year horizon
current_prices = adj_close.iloc[-1]
future_prices = current_prices * np.exp(sim_residuals / 100) # Reversing GARCH scale

# STEP 6: PRICING THE WORST-OF OPTION
print("Pricing the Worst-of Structured Product...")
# Performance = (Future Price / Current Price)
sim_performance = future_prices / current_prices

# The 'Worst-of' is the minimum performance in each simulation
worst_of_perf = sim_performance.min(axis=1)

# Define product: Pays the investor if the worst stock drops below 90%
strike = 0.90
payoffs = np.maximum(0, strike - worst_of_perf)

# The fair price is the average payoff across all simulations
price_estimate = payoffs.mean()

print("\n" + "="*40)
print(f"FINAL STRUCTURED PRODUCT RESULTS")
print(f"Underlying Assets: {tickers}")
print(f"Protection Level (Strike): {strike:.0%}")
print(f"Estimated Option Price: {price_estimate:.4%}")
print("="*40)

[*********************100%***********************]  3 of 3 completed

Fitting GARCH models to filter volatility clusters...


Modeling fat tails and transforming to Uniform [0,1]...
Calibrating StudentCopula for tail dependence...
              AAPL         MSFT        GOOGL
count  1257.000000  1257.000000  1257.000000
mean      0.500000     0.500000     0.500000
std       0.288560     0.288560     0.288560
min       0.000795     0.000795     0.000795
25%       0.250397     0.250397     0.250397
50%       0.500000     0.500000     0.500000
75%       0.749603     0.749603     0.749603
max       0.999205     0.999205     0.999205
Step 5: Generating 100000 correlated scenarios...
Scenarios generated. Proceeding to final price calculation...
Pricing the Worst-of Structured Product...

FINAL STRUCTURED PRODUCT RESULTS
Underlying Assets: ['AAPL', 'MSFT', 'GOOGL']
Protection Level (Strike): 90%
Estimated Option Price: 0.0001%
